In [5]:
import os
import asyncio
from openai import OpenAI
from dotenv import load_dotenv
from QwenIELTSEvaluator import QwenIELTSEvaluator
import whisper
from pathlib import Path
from utils import save_txt

In [ ]:
!pip install dotenv

In [6]:
load_dotenv()
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
testset_dir = Path("./testset")
if not testset_dir.exists():
    print("'testset' folder not found. Create it and add subfolders with MP3 files.")
    exit(1)
    
evaluator = QwenIELTSEvaluator(api_key=DASHSCOPE_API_KEY)
model = whisper.load_model("base") 
    
for subdir in testset_dir.iterdir():
    if not subdir.is_dir():
        continue  # Skip non-folders
        
    print(f"\n" + "="*50)
    print(f"Processing: {subdir.name}")
    print("="*50)
    
    base_url = f"https://github.com/hk414/automatic-feedback-for-ielts-speaking-asr/raw/refs/heads/main/testset/{subdir.name}/{subdir.name}_"
    part_one = f"{base_url}part_1.mp3"
    part_two = f"{base_url}part_2.mp3"
    part_three = f"{base_url}part_3.mp3"
    feedback = subdir / f"{subdir.name}_feedback.mp3"
    
    if not feedback.exists():
        print(f"Feedback MP3 not found for {subdir.name}: {feedback}")
        continue
    
    # Track results and errors
    results = {}
    errors = {}
    
    # Evaluate Part 1
    print("\nEvaluating Part 1...")
    try:
        p1_result = evaluator.evaluate_audio(part_one)
        results['part1'] = p1_result
        if p1_result:
            save_txt(p1_result, str(subdir / "part1_feedback.txt"))
    except Exception as e:
        error_msg = f"Error in Part 1: {str(e)}"
        print(f"{error_msg}")
        errors['part1'] = error_msg
        results['part1'] = None
    
    # Evaluate Part 2
    print("\nEvaluating Part 2...")
    try:
        p2_result = evaluator.evaluate_audio(part_two)
        results['part2'] = p2_result
        if p2_result:
            save_txt(p2_result, str(subdir / "part2_feedback.txt"))
    except Exception as e:
        error_msg = f"Error in Part 2: {str(e)}"
        print(f"{error_msg}")
        errors['part2'] = error_msg
        results['part2'] = None
    
    # Evaluate Part 3
    print("\nEvaluating Part 3...")
    try:
        p3_result = evaluator.evaluate_audio(part_three)
        results['part3'] = p3_result
        if p3_result:
            save_txt(p3_result, str(subdir / "part3_feedback.txt"))
    except Exception as e:
        error_msg = f"Error in Part 3: {str(e)}"
        print(f"{error_msg}")
        errors['part3'] = error_msg
        results['part3'] = None
    
    # Check if we have at least one successful result
    successful_parts = [k for k, v in results.items() if v is not None]
    
    if not successful_parts:
        print("\nALL PARTS FAILED - Skipping evaluation for this folder")
        continue
    
    # Combine successful feedback
    print(f"\nSuccessfully evaluated: {', '.join(successful_parts)}")
    if errors:
        print(f"Failed parts: {', '.join(errors.keys())}")
    
    # Build combined feedback from successful parts only
    feedback_parts = []
    if results['part1']:
        feedback_parts.append(f"Part 1:\n{results['part1']}")
    if results['part2']:
        feedback_parts.append(f"Part 2:\n{results['part2']}")
    if results['part3']:
        feedback_parts.append(f"Part 3:\n{results['part3']}")
    
    generated_feedback = "\n\n---\n\n".join(feedback_parts)
    
    output_file1 = subdir / "model_feedback.txt"
    save_txt(generated_feedback, str(output_file1))
    
    model_feedback_dir = testset_dir / "model_feedback"
    output_file2 = model_feedback_dir / f"{subdir.name}.txt"
    save_txt(generated_feedback, str(output_file2))


Processing: -awN8jpvDFo_7_0

Evaluating Part 1...
Evaluating audio: https://github.com/hk414/automatic-feedback-for-ielts-speaking-asr/raw/refs/heads/main/testset/-awN8jpvDFo_7_0/-awN8jpvDFo_7_0_part_1.mp3

Error: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: Failed to download multimodal content', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-97e0fcf3-dcf9-9c0e-b662-98ca0b756467', 'request_id': '97e0fcf3-dcf9-9c0e-b662-98ca0b756467'}

Evaluating Part 2...
Evaluating audio: https://github.com/hk414/automatic-feedback-for-ielts-speaking-asr/raw/refs/heads/main/testset/-awN8jpvDFo_7_0/-awN8jpvDFo_7_0_part_2.mp3

Error: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: Failed to download multimodal content', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-97c10a84-4a81-9658-88ca-44bbba8f0109', 'request_id': '97c1

KeyboardInterrupt: 